In [1]:
import sys
import os
sys.path.append("./../bin/Release")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
goptimum_cec22 = {
    1 : 300, 2: 400, 3: 600, 4: 800, 5:900, 6:1800, 7:2000, 8:2200, 9:2300, 10: 2400, 11:2600, 12: 2700
}
goptimum_cec20 = {
    1 : 100, 2: 1100, 3: 700, 4: 1900, 5:1700, 6:1600, 7:2100, 8:2200, 9:2400, 10: 2500
}
goptimum_cec17 = { i:100*i for i in range(1, 31)}
goptimum_cec20 = np.array(list(goptimum_cec20.values()))
goptimum_cec22 = np.array(list(goptimum_cec22.values()))
goptimum_cec17 = np.array(list(goptimum_cec17.values()))

resFolder = "./../bin/"

# CEC 2017

In [44]:
filename = resFolder+"results_2017_NLSHADE_RSP_10_1000000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_10_1000000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_j2020_10_100000.txt"
#data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_10_1000000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_jSO_10_100000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_LSRTDE_10_100000.txt"
#data5 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_JADE_10_100000.txt"
#data6 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
   # "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    #"LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })
df

,NLSHADE_RSP,ARRDE,LSHADE
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,0.047381,2.226905,1.470524
5,0.000000,0.000000,0.000000
6,8.119810,11.195810,11.621190
7,0.000000,1.658333,1.042429
8,0.000000,0.000000,0.000000
9,8.045238,31.515238,6.464286


In [29]:
filename = resFolder+"results_2017_NLSHADE_RSP_30_300000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_30_300000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_30_300000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_30_300000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_jSO_30_300000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_30_300000.txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0),
    "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0)
    })
df

,NLSHADE_RSP,ARRDE,j2020,LSHADE,jSO,LSRTDE
0,0.000000,0.000000,0.000000e+00,0.0000,0.000000,0.000000
1,0.000182,0.000000,4.957335e+16,0.0000,0.000000,0.000000
2,136.880545,0.000000,6.165450e+02,0.0000,0.000000,0.000000
3,1.146636,16.140095,8.112418e+01,58.5620,43.820455,58.562000
4,60.444000,23.451524,5.966091e+01,13.6068,24.243182,2.623182
5,0.000000,0.000000,0.000000e+00,0.0000,0.000000,0.000000
6,86.800818,46.673000,9.966000e+01,43.8230,48.738273,43.319909
7,70.543455,20.710952,6.516227e+01,15.6018,26.955727,2.261364
8,11.004182,0.004286,0.000000e+00,0.0000,0.016273,0.000000
9,1801.646364,1855.609048,3.778100e+03,1838.9660,2166.723636,248.191818


In [53]:
filename = resFolder+"results_2017_NLSHADE_RSP_50_500000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_50_500000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_50_500000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_50_500000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_jSO_50_500000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_50_500000.txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
     "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
     "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0)
    })
df

,NLSHADE_RSP,ARRDE,j2020,LSHADE,jSO,LSRTDE
0,5.197871e+02,0.000000,4.195854e+03,0.0000,0.000000,0.000000
1,1.182972e+06,0.000000,5.351149e+43,0.0000,0.000000,0.000000
2,8.216470e+03,0.000091,4.841795e+04,0.0000,0.000000,0.000000
3,7.365936e+01,28.749545,9.640982e+01,85.4710,55.736273,33.710182
4,1.586380e+02,57.707545,1.489918e+02,27.3348,57.440636,5.698636
5,1.909091e-03,0.000000,0.000000e+00,0.0000,0.022818,0.000000
6,1.856759e+02,93.892909,2.370504e+02,70.2136,93.394909,89.291909
7,1.581315e+02,52.461636,1.539458e+02,30.0428,62.322182,4.703636
8,1.166579e+03,0.048909,8.181818e-03,0.0000,0.089818,0.000000
9,3.816497e+03,3720.556364,7.838334e+03,3068.8980,3856.119091,458.529091


In [54]:
filename = resFolder+"results_2017_NLSHADE_RSP_100_1000000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_100_1000000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2017_j2020_100_1000000.txt"
#data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_100_1000000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_jSO_100_1000000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_100_100000.txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    #"j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
     "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
     "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0)
    })
df

,NLSHADE_RSP,ARRDE,LSHADE,jSO,LSRTDE
0,1.533793e+03,0.0000,9.834000e-01,0.0000,6.259800e+79
1,2.994063e+27,785410.5890,2.895750e+09,562429.2032,6.259800e+79
2,7.390660e+04,3.4472,4.000000e-04,0.0008,6.259800e+79
3,1.541858e+02,116.6400,1.097256e+02,190.5850,6.259800e+79
4,5.255018e+02,131.7326,1.430774e+02,175.7328,6.259800e+79
5,4.800000e-03,0.0002,1.228000e-01,0.1250,6.259800e+79
6,5.434080e+02,241.7690,2.585078e+02,289.8502,6.259800e+79
7,5.312080e+02,134.9162,1.474548e+02,202.5742,6.259800e+79
8,2.194252e+04,1.3654,2.275170e+02,9.2574,6.259800e+79
9,1.032740e+04,10987.3000,1.002448e+04,10080.2200,6.259800e+79


# CEC 2020

In [37]:
maxevals=50000
filename = resFolder+"/results_2020_NLSHADE_RSP_5_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_5_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_5_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_5_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })
df

,NLSHADE_RSP,ARRDE,j2020,LSRTDE
0,0.000000,0.000000,0.000000,0.000000
1,0.132941,9.445882,13.137255,11.947255
2,2.903706,5.034529,4.654431,5.191804
3,0.002549,0.103333,0.086078,0.441961
4,0.000000,0.068039,0.123137,8.821569
5,0.000000,0.019804,0.000000,0.073529
6,0.000000,0.000000,0.000000,4.793922
7,0.000000,0.000000,2.904118,12.036667
8,3.945882,9.314118,54.398431,125.204118
9,234.653137,156.862745,122.852353,344.585686


In [36]:
maxevals=1000000
filename = resFolder+"results_2020_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })
df

,NLSHADE_RSP,ARRDE,j2020,LSRTDE
0,0.000000,0.000000,0.000000,0.000000
1,2.162857,6.679524,2.881429,3.558571
2,8.125048,11.432000,7.791095,11.015476
3,0.001429,0.324286,0.079048,0.693333
4,1.368571,1.883333,0.333333,2.058571
5,0.096667,0.213810,0.567619,0.405238
6,0.001905,0.044286,0.016667,0.592381
7,5.759048,3.966190,3.853333,95.251905
8,73.212857,52.380952,100.000000,281.292857
9,326.855238,218.948571,194.656667,408.731905


In [48]:
maxevals=3000000
filename = resFolder+"results_2020_NLSHADE_RSP_15_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_15_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2020_j2020_15_"+str(maxevals)+".txt"
#data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2020_LSRTDE_10_"+str(maxevals)+".txt"
#data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    #"j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
    #"LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })
df

,NLSHADE_RSP,ARRDE
0,0.000,0.0000
1,5.276,2.1760
2,15.567,16.1298
3,0.002,0.4440
4,16.904,10.8900
5,0.102,0.7180
6,0.298,0.3680
7,2.472,62.4720
8,80.000,100.0000
9,400.000,400.0000


In [47]:
maxevals=10000000
filename = resFolder+"results_2020_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2020_j2020_20_"+str(maxevals)+".txt"
#data2 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2020_LSRTDE_20_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    #"j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSRTDE" : np.mean(np.abs(data3-goptimum_cec20), axis=0)
    })
df

,NLSHADE_RSP,ARRDE
0,0.000000,0.0000
1,0.070000,0.9150
2,20.387000,20.4425
3,0.000000,0.5800
4,90.220000,23.0600
5,0.210000,0.1550
6,0.186667,0.5300
7,90.420000,100.0000
8,100.000000,100.0000
9,399.063333,413.6600


# CEC 2022


## 10D

In [56]:
maxevals=1000000
filename = resFolder+"results_2022_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22), axis=0),
    })
df

,NLSHADE_RSP,ARRDE,j2020,LSRTDE
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,1.709238
2,0.000000,0.000000,0.000000,0.000000
3,9.997048,2.511190,4.309000,0.284286
4,0.000000,0.000000,0.000000,0.000000
5,0.165714,0.330476,0.140952,0.422381
6,0.000000,0.000000,0.000000,0.265714
7,0.017143,0.122857,0.362381,0.213333
8,218.361905,196.525714,192.342857,266.238095
9,0.002857,9.704762,0.000000,105.116190


## 20D


In [63]:
maxevals=1000000
filename = resFolder+"results_2022_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2022_j2020_20_"+str(maxevals)+".txt"
#data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_LSHADE_20_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_jSO_20_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_20_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    #"j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0)
     "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22), axis=0), 
    })
df

,NLSHADE_RSP,ARRDE,LSRTDE
0,0.000000,0.000000,0.000000
1,43.512476,6.854857,44.895000
2,0.000000,0.000000,0.000000
3,69.180667,11.465810,0.852857
4,0.000000,0.000000,0.000000
5,17.538571,1.056190,0.513333
6,8.627619,16.449048,0.342857
7,20.504286,19.626190,19.054286
8,180.780000,180.780000,180.780000
9,0.000000,0.113810,105.619048
